# Experiment: _Node weights_

In [ ]:
from notebook_prelude import *

In [ ]:
df = results_helper.get_experiments_by_names(['experiment_node_weights', 'experiment_graphs_plain'], fetch_predictions=True)
for x in ['graph__fast_wl__node_weight_iteration_weight_function', 'graph__fast_wl__node_weight_iteration_weight_function']:
    df[x + '_'] = df[x].apply(lambda x: x.__name__ if callable(x) else '-')
df = df[df.type != TYPE_COOCCURRENCE]

In [ ]:
attr = 'mean_test_f1_macro'
attr = 'prediction_score_f1_macro'
df_ = df.groupby(['dataset', 'graph__fast_wl__node_weight_function'])[attr].max().to_frame().unstack()
df_['difference_non_vs_weight'] = (df_[attr, 'none'] - df_[attr, 'nxgraph_degrees_metric'])
df_

In [ ]:
df_confidence = significance_test_utils.get_confidences(df[df.graph__fast_wl__node_weight_function != 'nxgraph_degrees_metric_max'], performance_attr='prediction_score_f1_macro', model_selection_attr='graph__fast_wl__node_weight_function', model_selection_vals=['nxgraph_degrees_metric', 'none'], log_progress=log_progress_nb)

In [ ]:
df_['confidence'] = df_.index.map(lambda x: df_confidence.loc[x].confidence)
df__ = df_[[
    ('prediction_score_f1_macro', 'none'),
    ('prediction_score_f1_macro', 'nxgraph_degrees_metric_max'),
    ('confidence', '')
]]
print(df__.to_latex(float_format=lambda x: '%.4f' % x))
df__

## Other graph datasets

_Mutag_, _Enzymes_, etc.

In [ ]:
df = results_helper.get_experiments_by_names(['experiment_graph_extra_plain', 'experiment_graph_extra_node_weights'], fetch_predictions=True)
df['graph__fast_wl__node_weight_function'] = df.graph__fast_wl__node_weight_function.apply(lambda x: x.__name__ if callable(x) else x)

In [ ]:
pd.options.display.float_format = lambda x: '%.2f' % x
#attr = 'prediction_score_f1_macro'
attr = 'mean_test_f1_macro'
attr = 'prediction_score_f1_macro'
#attr = 'prediction_score_accuracy'
best = df.groupby(['dataset', 'graph__fast_wl__node_weight_function'])[attr].max().to_frame().unstack()
best.columns = best.columns.droplevel()
best.rename(columns={'-': 'plain'}, inplace=True)
best = best[(best.none != '-') & (best.adj_degrees_metric_max != '-')]
best = best[best.adj_degrees_metric_max != None]
best['difference'] = best.adj_degrees_metric_max - best.none
#print(best.to_latex())
best

### Statistics about benchmark graphs

In [ ]:
data = collections.defaultdict(list)
for dataset in log_progress_nb(graph_helper.get_all_graph_benchmark_dataset_names()):
    X, Y = graph_helper.get_mutag_enzyme_graphs(dataset)
    assert len(X) == len(Y)
    nodes = list(chain.from_iterable([labels for _, labels in X]))
    unique_nodes = set(nodes)
    num_edges = sum([adj.nonzero()[0].shape[0] for adj, labels in X])
    data['dataset'].append(dataset)
    data['num_nodes'].append(len(nodes))
    data['num_unique_nodes'].append(len(unique_nodes))
    data['num_edges'].append(num_edges)
    data['num_graphs'].append(len(X))
    data['num_classes'].append(len(set(Y)))

In [ ]:
groupby

In [ ]:
df.graph__fast_wl__node_weight_function.unique()

In [ ]:
attr = 'prediction_score_f1_macro'
df_graph_statistics = pd.DataFrame(data).set_index('dataset')
df_graph_statistics[df_graph_statistics.num_graphs > 1000]
filtered = df[(np.isin(df.dataset.values, df_graph_statistics[df_graph_statistics.num_graphs > 1000].index.values))]
best_ = filtered.groupby(groupby)[attr].max().to_frame().unstack()
groupby_attr = 'graph__fast_wl__node_weight_function'
best_ = best_[[
    (attr, 'none'),
    (attr, 'adj_degrees_metric_max')
]]
best_['difference'] = best_[attr, 'none'] - best_[attr, 'adj_degrees_metric_max']
best_

In [ ]:
attr

In [ ]:
filtered.graph__fast_wl__node_weight_function.unique()

In [ ]:
filtered['prediction_score_f1_macro'] = filtered.prediction_score_f1_macro.astype(np.float64)
filtered = filtered[filtered.graph__fast_wl__node_weight_function != 'adj_degrees_metric']
df_confidence = significance_test_utils.get_confidences(filtered, performance_attr=attr, model_selection_attr='graph__fast_wl__node_weight_function', model_selection_vals=['adj_degrees_metric_max', 'none'], log_progress=log_progress_nb)

In [ ]:
best_['confidence'] = best_.index.map(lambda x: df_confidence.loc[x].confidence)

print(best_.drop(columns='difference').to_latex(float_format=lambda x: '%.4f' % x))
best_